In [264]:
import igraph as ig
import csv
from collections import defaultdict
import community
import networkx as nx
import numpy as np
import operator
import pydot
import plotly.plotly as py
import plotly.graph_objs as go
from networkx.drawing.nx_pydot import write_dot
from networkx.drawing.nx_pydot import graphviz_layout
from networkx.drawing.nx_pydot import pydot_layout
import matplotlib.pyplot as plt
import matplotlib
import plotly
import plotly.offline as pyo
import math
import random
plotly.tools.set_credentials_file(username='pcsplotly', api_key='ofJJAPLfVgSYzk4mmyzi')

filepath = "project_companies/"
louvainoutput = "louvain_clean_names.csv"
overlapgraph = "overlapgraph_clean"

In [7]:
ig.__version__  #requires version 0.7.1

'0.7.1'

In [323]:
nx.__version__ #requires version 1.11

'1.11'

In [260]:
plotly.__version__ #requires 2.7.0

'2.7.0'

# Loading the Louvain Output 
## and convert it into a dictionary of node:partition

In [3]:
##Building the main dictionary from the louvainoutput file
threshold = 200
belowthreshold = []
sizes = defaultdict()
partitioning = defaultdict()
print(filepath+louvainoutput)
with open(filepath+louvainoutput, "rt", encoding="utf8") as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    for row in reader:
        partition=row.pop(0)
        size=int(row.pop(0))
        if size < threshold:
                belowthreshold.append(partition)
        else:
            sizes[partition] = size
        for node in row:
            partitioning[node] = partition
           

project_companies/louvain_clean_names.csv


## Loading the overlapgraph as a proper nx graph

In [4]:
def buildWeightedGraph():
    print("Building weighted graph from %s..." % (filepath+overlapgraph))
    fh = open(filepath+overlapgraph, 'rb')
    G = nx.read_weighted_edgelist(fh, delimiter=";")
    print("Done.")
    return G

if True:
    original = buildWeightedGraph()

Building weighted graph from project_companies/overlapgraph_clean...
Done.


### check if overlapgraph and partitioning are consitent

In [5]:
membersinoverlap = list(map(int, original.nodes()))
uniqueinoverlap = list(set(membersinoverlap) - set(map(int,partitioning.keys())))
uniqueinclustrering = list(set(map(int,partitioning.keys())) - set(membersinoverlap))
bool(uniqueinclustrering) == bool(uniqueinoverlap) == False

True

## Build the induced graph based on the partitioning and the original graph

In [6]:
## get induced graph for all clusters
if False:
    induced = community.induced_graph(partitioning, original)
    # Save induced graph
    nx.write_gpickle(induced, filepath+"clustering_induced_graph.pkl")	
else:
    induced = nx.read_gpickle(filepath+"clustering_induced_graph.pkl")	

In [8]:
# removing too small clusters
induced.remove_nodes_from(belowthreshold)

In [9]:
sorted_sizes=[x for _,x in sorted(sizes.items(), key=lambda kv: kv[1], reverse=True)]
max_size = max(sorted_sizes)
min_size = min(sorted_sizes)

def normalize(value, max_size=max_size, min_size=min_size):
    return (value - min_size) / (max_size-min_size)

normalized_sizes = defaultdict()
for key,value in sizes.items():
    normalized_sizes[key]=normalize(value)

In [11]:
# adding the size attribute
nx.set_node_attributes(induced, "size", sizes)

# adding the normalized size attribute
nx.set_node_attributes(induced, "normalized_size", normalized_sizes)

# adding current (louvain) name as attribute
nx.set_node_attributes(induced, 'louvain_name', dict(zip( induced.nodes(),  induced.nodes())))

In [318]:
# size mapping <oldname><sizerank>  /ranking by size and relabelling
mapping = dict(zip([x for x,_ in sorted(sizes.items(), key=lambda kv: kv[1], reverse=True)], range(0, induced.number_of_nodes())))
induced_relabeled = nx.relabel_nodes(induced, mapping)

In [321]:
# export to .dot
nx.write_graphml(induced_relabeled,filepath+'induced.graphml') # Export NX graph to file
induced_ig = ig.read(filepath+'induced.graphml',format="graphml") # Create new IG graph from file

In [322]:
# export to .dot
nx.write_pajek(induced_relabeled,filepath+'induced.pajek') # Export NX graph to file
induced_ig = ig.read(filepath+'induced.pajek',format="pajek") # Create new IG graph from file

SystemError: <built-in method Read_Pajek of type object at 0x0000022C75659EA8> returned NULL without setting an error

# Plotting Nx Graph in 2D

In [309]:
# generate positions for the nodes (standard layout)
pos=pydot_layout(induced_relabeled,prog='dot')

# generate positions for the nodes (spring/force layout)
force_pos = {key:(value[0],value[1]) for key,value in nx.spring_layout(induced_relabeled,scale=10000).items()}

# generate positions for the nodes (spectral layout)
circular_pos = {key:(value[0],value[1]) for key,value in nx.circular_layout(induced_relabeled,scale=10000).items()}

# add these positions as node features
nx.set_node_attributes(induced_relabeled, "pos", circular_pos)

In [310]:
# calculate general position infos on the whole graph
dmin=1
ncenter=0
for n in pos:
    x,y=pos[n]
    d=(x-0.5)**2+(y-0.5)**2
    if d<dmin:
        ncenter=n
        dmin=d
        
p=nx.single_source_shortest_path_length(induced_relabeled,ncenter)

In [311]:
edge_weights = []

for u,v in induced_relabeled.edges():
    if u != v:
        edge_weights.append(math.log(induced_relabeled.get_edge_data(u,v)["weight"]+1))
    else:
        edge_weights.append(0)
        
max_log_edgeweight=max(edge_weights)

In [312]:
edge_weights = norm = [float(i)/max(edge_weights)*5 for i in edge_weights]


## Create Edges

In [313]:
#edge_trace = go.Scatter(
#    x=[],
#    y=[],
#    line=dict(width=0.5,color='#888'),
#    text=dumm_edge_weights,
#    hoverinfo="text",
#    mode='lines')
#
#for edge in induced_relabeled.edges():
#    x0, y0 = induced_relabeled.node[edge[0]]['pos']
#    x1, y1 = induced_relabeled.node[edge[1]]['pos']
#    edge_trace['x'] += [x0, x1, None]
#    edge_trace['y'] += [y0, y1, None]

edge_traces = []

for edge,weight in zip(induced_relabeled.edges(),edge_weights):
    x0, y0 = induced_relabeled.node[edge[0]]['pos']
    x1, y1 = induced_relabeled.node[edge[1]]['pos']
    edge_traces.append(go.Scatter(
    x=[x0, x1, None],
    y=[y0, y1, None],
    line=dict(width=weight,color='#888'),
    text = str(weight),
    hoverinfo ='text',
    mode='lines'))


node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
        # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
        colorscale='YIGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

for node in induced_relabeled.nodes():
    x, y = induced_relabeled.node[node]['pos']
    node_trace['x'].append(x)
    node_trace['y'].append(y)

## Manipulate Edge Appearance

## Create Node Colors

In [314]:
#individual node setting
colors = []
descriptions = []
node_sizes = []
sizefactor = 50
for node, adjacencies in enumerate(induced_relabeled.adjacency_list()):
    colors.append(len(adjacencies))
    node_info = 'Node no: '+str(node)+'<br>Size: '+str(induced_relabeled.node[node]['size'])+'<br># of connections: '+str(len(adjacencies))+'<br>Louvain id: '+str(induced_relabeled.node[node]['louvain_name'])
    descriptions.append(node_info)
    #node_sizes = list(map(lambda v : (v+1)*sizefactor,normalized_sizes))
    node_sizes.append((induced_relabeled.node[node]['normalized_size']+1)*sizefactor)
    
node_trace['marker']['color'] = colors
node_trace['text'] = descriptions
node_trace['marker']['size'] = node_sizes
    

## Create/Draw Network Graph


In [315]:
fig = go.Figure(data=[*edge_traces, node_trace],
             layout=go.Layout(
                title='<br>Network graph',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="...",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

py.iplot(fig, filename='networkx')
#pyo.plot(fig, filename='networkx')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~pcsplotly/0 or inside your plot.ly account where it is named 'networkx'


<br><br><br><br>
# igraph

In [252]:
N = len(induced_relabeled.nodes())
#layt=induced_ig.layout('circular_3d')
layt=induced_ig.layout('kk',dim=3)

In [253]:
scale=1

Xn=[layt[k][0]*scale for k in range(N)]# x-coordinates of nodes
Yn=[layt[k][1]*scale  for k in range(N)]# y-coordinates
Zn=[layt[k][2]*scale  for k in range(N)]# z-coordinates
Xe=[]
Ye=[]
Ze=[]


#for e in induced_ig.get_edgelist():
#    Xe+=[layt[e[0]][0],layt[e[1]][0], None]# x-coordinates of edge ends
#    Ye+=[layt[e[0]][1],layt[e[1]][1], None]
#    Ze+=[layt[e[0]][2],layt[e[1]][2], None]

In [254]:
#new single edge traces with weights and annotations
edge_traces_3d = []
    
for edge in induced_ig.es:
    source_vertex_id = edge.source
    target_vertex_id = edge.target
    Xe=[layt[source_vertex_id][0],layt[target_vertex_id][0], None]# x-coordinates of edge ends
    Ye=[layt[source_vertex_id][1],layt[target_vertex_id][1], None]
    Ze=[layt[source_vertex_id][2],layt[target_vertex_id][2], None]
    if source_vertex_id == target_vertex_id:
        weight = 0
    else:
        weight = (math.log(edge['weight'])/max_log_edgeweight)*10
        #weight = ((edge['weight'])/math.exp(max_log_edgeweight))*10
        #weight = random.random()*50
        
    edge_traces_3d.append(go.Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               line=dict(color='rgb(125,125,125)', 
               width=weight),
               text=str(round(edge['weight'])),
               hoverinfo='none'
               ))

In [255]:
##Original edge trace
#trace1=go.Scatter3d(x=Xe,
#               y=Ye,
#               z=Ze,
#               mode='lines',
#               line=dict(color='rgb(125,125,125)', width=1),
#               hoverinfo='none'
#               )

#node trace
node_trace_3d =go.Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               name='clusters',
               marker=dict(symbol='dot',
                             size=[x/2 for x in node_sizes],
                             color=colors,
                             colorscale='Viridis',
                             line=dict(color='rgb(50,50,50)', width=0.5)
                             ),
               text=descriptions,
               hoverinfo='text'
               )

#heading trace
label_trace_3d =go.Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='text',
               text="Label",
               hoverinfo='none'
               )

In [257]:
axis=dict(showbackground=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title='')

In [300]:
#node_sizes

In [258]:
layout = go.Layout(
         title="Network (3D visualization)",
         width=1600,
         height=900,
         showlegend=False,
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
        ),
     margin=dict(
        t=100
    ),
    hovermode='closest',
    annotations=[
           dict(
           showarrow=False,
            text="...",
            xref='paper',
            yref='paper',
            x=0,
            y=0.1,
            xanchor='left',
            yanchor='bottom',
            font=dict(
            size=14
            )
            )
        ],    )

In [259]:
data=[*edge_traces_3d, node_trace_3d,label_trace_3d]
fig=go.Figure(data=data, layout=layout)


#py.iplot(fig, filename='3D')
pyo.plot(fig, filename='3D')

c:\python35\lib\site-packages\plotly\offline\offline.py:459: UserWarning:

Your filename `3D` didn't end with .html. Adding .html to the end of your file.



'file://D:\\WikiData\\3D.html'